The MIT License

Copyright (c) 2016 OpenAI (http://openai.com)

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

# Using evolution strategies to train Roboschool Environments

## Introduction

This Jupyter Notebook is based on the [paper](https://arxiv.org/abs/1703.03864), [blog article](https://openai.com/blog/evolution-strategies/) and [implementation](https://github.com/openai/evolution-strategies-starter) of OpenAI on the topic of using an evolution strategy algorithm for a typical reinforcement learning task. 

My implementation summarizes their implementation, by simplifying, refactoring and organizing the code into this Jupyter notebook which can be used to test the algorithm. One can tweak the hyperparameters, change the environment which shall be trained or even expand the implementation to support for example Atari environments.

I recommend reading the paper or at least the article before trying out the notebook. Also depending on the environment the training can be very computationally intense (for example training the Humanoid), so if you want to try out the harder ones I recommend using a highly parallelizable machine, i.e. a machine with a high number of cores/threads.

## Algorithm overview

This section gives a brief overview over the used strategy. First of all we need to define what this implementation is going to do. The Roboschool is a group of environments in the [OpenAi Gym](https://gym.openai.com/), a program to test the behavior of machine learning algorithms on _real world_ problems. In our case, we want to train different robotic environments using evolution strategies. We therefore define a neural net with a configurable number of hidden layers, where the input dimension equals the observation space of the environment and the dimension of the output layer equals the dimension of the action space of the environment. This neural net will be called policy in our context. Therefore we train our policy to output the best possible action sequence given an observation sequence. Now, how do we train this policy? Training an evolutionary strategy consists of a cycle which is repeated over and over. First, an initial weight vector is randomly generated. Then we perturb this vector. The number of perturbations is called the population size. Lets say we have a population size of 100. Therefore we now have 100 weight vectors which are slightly different from the initial one. We then take these new weight vectors and evaluate them on our environment. The environment gives us a reward back, a number which indicates how well the policy with our perturbed weight vectors has done. So now we have 100 perturbed weight vectors with 100 noise vectors and 100 rewards for each perturbed weight vector. We then sum over all the rewards and weight them with the used noise. We then multiply this sum with the learning rate and divide by the population size and the noise standard deviation. The result of this equation gives us the new step which we add to the inital weight vector. Then we have the initial weight vector for the next cycle. We call such a cycle a generation. What it essentially does is evaluate which noise we added resulted in the best reward and we want the next generation to base on that noise.

## Setup

Before starting any computation we need to configure the program and define some methods and objects we will use later on.


### Imports

Note that here is not import of TensorFlow.

In [ ]:
import errno
import logging
import os
import time

from collections import namedtuple

import ctypes
import multiprocessing
import numpy as np

import gym, roboschool

### Logging and save directory

First things first, we need to define a directory where we want to store the trained weights, as well as the log file to record the results of every generation.

The default path a folder called `es_{PID}` inside the working directory of this Jupyter Notebook, where the PID is the Process ID. If you want to change this, set save_directory to a valid path of your choice in a non-root location.

In [ ]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

save_directory = os.getcwd() + "/es_{}/".format(os.getpid())
mkdir_p(save_directory)

logger = logging.getLogger('main_logger')
logger.setLevel(logging.INFO)

fh = logging.FileHandler(save_directory + 'log.txt')
fh.setLevel(logging.INFO)
fh.setFormatter(logging.Formatter('%(asctime)s %(message)s', ''))

logger.addHandler(fh)

### Configuration & Result Classes

Using a `namedtuple` allows use to quickly create a class with different attributes, which is ideal for defining a Config and Result Class.

#### Config Class

The Config class defines our general configuration of the program. The following table explains what each attribute does:


| Attribute             | Explanation   |
| :---------------------|:--------------|
| `env_id`              | A string representing an environment of the Roboschool, e.g. `"RoboschoolAnt-v1"`|
| `population_size`     | The size of the population of one generation    |
| `num_workers`         | The number of workers doing computation in parallel, for maximum performance the default is `os.cpu_cores`|
| `learning_rate`       | A floating point number which dictates how _strong_ the best candidate of a generation shall be influencing the paramter vector 
| `noise_stdev`         | The standard deviation used for the noise
| `snapshot_freq`       | An integer which tells in which frequency the weights of the model shall be saved, e.g. a snapshot frequency of 10 would save the weights of the model every 10 generations
| `return_proc_mode`    | The processing mode of the gathered returns, currentyl only `"centered_rank"` is available
| `timesteps_per_gen` | Alternative exit condition for a generation, i.e. stop after x timesteps, currently not used
| `calc_obstat_prob`    | The probability of calculating the new mean and standard deviation of the observation space
| `eval_prob`           | The probability of inserting an evaluation result

In [ ]:
Config = namedtuple('Config', [
    'env_id',
    'population_size',
    'num_workers',
    'learning_rate',
    'noise_stdev',
    'snapshot_freq',
    'return_proc_mode',
    #'timesteps_per_gen',
    'calc_obstat_prob',
    'l2coeff'
    #'eval_prob'
])

In [ ]:
Optimizations = namedtuple('Optimizations', [
    'mirrored_sampling',
    'fitness_shaping',
    'weight_decay', # TODO
    'discretize_actions', # TODO how to save model with the output?
    'neural_network_optimizer',
    'observation_space_normalization'
])

#### Model Structure class

| Attribute         | Explanation   |
| :---------------- |:--------------|
| `ac_noise_std`    |               |
| `connection_type` |               |
| `hidden_dims`     |               |
| `nonlin_type`     |               |

In [ ]:
ModelStructure = namedtuple('ModelStructure', [
    'ac_noise_std',
    'ac_bins',
    'connection_type',
    'hidden_dims',
    'nonlin_type',
    'nn_optimizer',
    'nn_optimizer_args'
])

#### Result class

| Attribute             | Explanation   |
| :---------------------|:--------------|
| `noise_inds_n`        | |
| `returns_n2` |
| `signreturns_n2` |
| `lengths_n2` |
| `eval_return` |
| `eval_length` |
| `ob_sum` |
| `ob_sumsq` |
| `ob_count` |
| `task_id` | 



In [ ]:
Result = namedtuple('Result', [
    'noise_inds','returns', 'signreturns', 'lengths',
    'eval_return', 'eval_length',
    'ob_sum', 'ob_sumsq', 'ob_count',
    'task_id'
])

## Config

In [ ]:
config = Config(
    env_id="RoboschoolHumanoid-v1",
    population_size=1000,
    num_workers=os.cpu_count(),
    learning_rate=0.001,
    noise_stdev=0.02,
    snapshot_freq=1,
    return_proc_mode="centered_rank",
    calc_obstat_prob=0.01,
    l2coeff=0.005
)

## Environment

Create one for every worker -> done in worker method
Master also

In [ ]:
env = gym.make(config.env_id)

## Policy setup

Currently saves the arguments as local variable, then creates a TensorFlow variable scope where the neural network
architecture gets created.

Currently emitted:
6. set_all_vars

## Keras as Model

Original implementation used hand written dense layers and tensorflow operations. I use a Keras model and their
functional API to create the net. In testing the two version differ in 0.x float scope. Something to worry about?

In [ ]:
model_structure = ModelStructure(
    ac_noise_std=0.01,
    ac_bins='continuous:',
    connection_type='ff',
    hidden_dims=[256, 256],
    nonlin_type='tanh',
    nn_optimizer='adam',
    nn_optimizer_args={
        'stepsize': 0.01
    }
)

ob_space= env.observation_space
ac_space = env.action_space
#ac_bins = args["ac_bins"] TODO ac_bins implementation

## Optimizations

In [ ]:
optimizations = Optimizations(
    mirrored_sampling=True,
    fitness_shaping=True,
    weight_decay=True,
    discretize_actions=False,
    neural_network_optimizer=True,
    observation_space_normalization=True
)

In [ ]:
class RunningStat(object):
    def __init__(self, shape, eps):
        self.sum = np.zeros(shape, dtype=np.float32)
        self.sumsq = np.full(shape, eps, dtype=np.float32)
        self.count = eps

    def increment(self, s, ssq, c):
        self.sum += s
        self.sumsq += ssq
        self.count += c

    @property
    def mean(self):
        return self.sum / self.count

    @property
    def std(self):
        return np.sqrt(np.maximum(self.sumsq / self.count - np.square(self.mean), 1e-2))

## Keras clearin backend to support multiprocessing

$\text{out } = \frac{\sigma}{\sqrt{\sum \limits_{i \in \text{out}} i^2}}$

In [ ]:
def create_model(initial_weights=None, model_name="model", save_path=None):
    import tensorflow as tf
    
    class Normc_initializer(tf.keras.initializers.Initializer):
        """
        Create a TensorFlow constant with random numbers normed in the given shape.
        :param std:
        :return:
        """
        def __init__(self, std=1.0):
            self.std=std

        def __call__(self, shape, dtype=None, partition_info=None):#pylint: disable=W0613
            out = np.random.randn(*shape).astype(np.float32)
            out *= self.std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
            return tf.constant(out)
    
    nonlin = tf.nn.tanh
    
    if model_structure.nonlin_type == 'relu':
        nonlin = tf.nn.relu
    elif model_structure.nonlin_type == 'lrelu':
        nonlin = tf.nn.leaky_relu
    elif model_structure.nonlin_type == 'elu':
        nonlin = tf.nn.leaky_relu
    

    # Policy network
    input_layer = x = tf.keras.Input(ob_space.shape, dtype=tf.float32)

    for hd in model_structure.hidden_dims:
        x = tf.keras.layers.Dense(
            hd, activation=nonlin,
            kernel_initializer=Normc_initializer(std=1.0),
            bias_initializer=tf.initializers.zeros())(x)

    # Map to action
    adim = ac_space.shape[0]

    a = tf.keras.layers.Dense(
        adim,
        kernel_initializer=Normc_initializer(std=1.0),
        bias_initializer=tf.initializers.zeros())(x)
    
    model = tf.keras.Model(inputs=input_layer, outputs=a, name=model_name)
    
    if initial_weights is not None:
        set_from_flat(model, initial_weights)
        
    if save_path:
        model.save(save_path)
        
    return model

In [ ]:
def act(ob, model, random_stream=None, ob_mean=None, ob_std=None):
    ob_proc = ob
    if ob_mean is not None and ob_std is not None:
        ob_proc = np.clip((ob - ob_mean) / ob_std, -5.0, 5.0)
    action = model.predict(ob)
    
    # TODO why randomstream? Better generalization?
    if random_stream is not None and model_structure.ac_noise_std != 0:
        action += random_stream.randn(*action.shape) * model_structure.ac_noise_std
    return action

def get_initial_weights():
    model = create_model()
    
    # Print out the model
    model.summary()
    
    return model.get_weights()

with multiprocessing.Pool(1) as pool:
    theta = pool.apply(func=get_initial_weights)

num_params = sum(np.prod(v.shape) for v in theta)

#placeholders = [tf.placeholder(v.value().dtype, v.get_shape().as_list()) for v in self.all_variables]

# self.set_all_vars = U.function(
#     inputs=placeholders,
#     outputs=[],
#     updates=[tf.group(*[v.assign(p) for v, p in zip(self.all_variables, placeholders)])]
# )

## Optimization: Using Adam Optimizer

Defining it manually since with Keras you have to define a loss function and use training set, etc. Manually seems
easier for now.

In [ ]:
class Optimizer(object):
    def __init__(self):
        self.dim = num_params
        self.t = 0

    def update(self, theta, globalg):
        self.t += 1
        step = self._compute_step(globalg)
        ratio = np.linalg.norm(step) / np.linalg.norm(theta)
        theta_new = theta + step
        return theta_new, ratio

    def _compute_step(self, globalg):
        raise NotImplementedError

class SGD(Optimizer):
    def __init__(self, stepsize, momentum=0.9):
        Optimizer.__init__(self)
        self.v = np.zeros(self.dim, dtype=np.float32)
        self.stepsize, self.momentum = stepsize, momentum

    def _compute_step(self, globalg):
        self.v = self.momentum * self.v + (1. - self.momentum) * globalg
        step = -self.stepsize * self.v
        return step
        
class Adam(Optimizer):
    def __init__(self, stepsize, beta1=0.9, beta2=0.999, epsilon=1e-08):
        Optimizer.__init__(self, theta)
        self.stepsize = stepsize
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = np.zeros(self.dim, dtype=np.float32)
        self.v = np.zeros(self.dim, dtype=np.float32)

    def _compute_step(self, globalg):
        a = self.stepsize * np.sqrt(1 - self.beta2 ** self.t) / (1 - self.beta1 ** self.t)
        self.m = self.beta1 * self.m + (1 - self.beta1) * globalg
        self.v = self.beta2 * self.v + (1 - self.beta2) * (globalg * globalg)
        step = -a * self.m / (np.sqrt(self.v) + self.epsilon)
        return step

In [ ]:
if optimizations.neural_network_optimizer:
    if model_structure.nn_optimizer == 'adam':
        optimizer = Adam(**model_structure.nn_optimizer_args)
    elif model_structure.nn_optimizer == 'sgd':
        optimizer = SGD(**model_structure.nn_optimizer_args)
    else:
        raise NotImplementedError

## Shared Noise

In [ ]:
class SharedNoiseTable(object):
    def __init__(self):
        seed = 123
        count = 250000000  # 1 gigabyte of 32-bit numbers. Will actually sample 2 gigabytes below.
        #logger.info('Sampling {} random numbers with seed {}'.format(count, seed))

        # Instantiate an array of C float datatype with size count
        self._shared_mem = multiprocessing.Array(ctypes.c_float, count)

        # Convert to numpy array
        self.noise = np.ctypeslib.as_array(self._shared_mem.get_obj())
        assert self.noise.dtype == np.float32
        self.noise[:] = np.random.RandomState(seed).randn(count)  # 64-bit to 32-bit conversion here
        #logger.info('Sampled {} bytes'.format(self.noise.size * 4))

    def get(self, i, dim):
        return self.noise[i:i + dim]

    def sample_index(self, stream, dim):
        return stream.randint(0, len(self.noise) - dim + 1)

noise = SharedNoiseTable()

## Get flat

In [ ]:
def get_flat(theta):
     return np.concatenate([np.reshape(v, [-1]) for v in theta], 0)

def set_from_flat(model, theta):
    old_theta = model.get_weights()
    shapes = [v.shape for v in old_theta]
    total_size = theta.size
    
    start = 0
    reshapes = []
    
    for (shape, v) in zip(shapes, theta):
        size = int(np.prod(shape))
        reshapes.append(np.reshape(theta[start:start+size], shape))
        start += size
    
    assert start == total_size
    model.set_weights(reshapes)
    

## Set from flat

## Rollout TODO

In [ ]:
def rollout(env, 
            model, 
            *, 
            render=False, 
            timestep_limit=None, 
            save_obs=False, 
            random_stream=None, 
            ob_mean=None, 
            ob_std=None):
    """
    If random_stream is provided, the rollout will take noisy actions with noise drawn from that stream.
    Otherwise, no action noise will be added.
    """
    
    env_timestep_limit = env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')
    timestep_limit = env_timestep_limit if timestep_limit is None else min(timestep_limit, env_timestep_limit)
    rews = []
    t = 0
    if save_obs:
        obs = []
    ob = env.reset()
    for _ in range(timestep_limit):
        ac = act(ob[None], model, random_stream=random_stream, ob_mean=ob_mean, ob_std=ob_std)[0]
        if save_obs:
            obs.append(ob)
        ob, rew, done, _ = env.step(ac)
        rews.append(rew)
        t += 1
        if render:
            env.render()
        if done:
            break
    rews = np.array(rews, dtype=np.float32)
    if save_obs:
        return rews, t, np.array(obs)
    return rews, t


# Worker method


In [ ]:
def rollout_and_update_ob_stat(env, model, rs, task_ob_stat, ob_mean=None, ob_std=None):
    if ob_mean is not None and ob_std is not None and config.calc_obstat_prob != 0 and rs.rand() < config.calc_obstat_prob:
        rollout_rews, rollout_len, obs = rollout(
            env, model, save_obs=True, random_stream=rs)
        task_ob_stat.increment(obs.sum(axis=0), np.square(obs).sum(axis=0), len(obs))
    else:
        rollout_rews, rollout_len = rollout(env, model, random_stream=rs)
    return rollout_rews, rollout_len

In [ ]:
def run_worker(num_jobs, theta, ob_mean=None, ob_std=None): #min_task_runtime=.2):

    print("PID " + str(os.getpid()) + ": " + "Started worker with " + str(num_jobs) + "Jobs")

    assert isinstance(noise, SharedNoiseTable)

    # Setup
    #config, env, sess, policy = setup(exp, single_threaded=True)
    env = gym.make(config.env_id)
    model = create_model(initial_weights=theta, model_name=str(os.getpid()))
    

    # Random stream used for todo
    rs = np.random.RandomState()

    #assert policy.needs_ob_stat == (config.calc_obstat_prob != 0)

    #while True:
    # Prevent accessing empty array (master did not emit task yet)
    #while not tasks:
    #    time.sleep(0.05)

    #task_data = tasks[-1]

    #task_tstart = time.time()

    #assert isinstance(task_data, Task)
    #task_id = task_data.task_id
    #assert isinstance(task_id, int)

    #if policy.needs_ob_stat:
    #    policy.set_ob_stat(task_data.ob_mean, task_data.ob_std)

    # # todo whats this condition doing?
    # if rs.rand() < config.eval_prob:
    #     # Evaluation: noiseless weights and noiseless actions
    #     policy.set_trainable_flat(task_data.params)
    # 
    #     eval_rews, eval_length = policy.rollout(env)  # eval rollouts don't obey task_data.timestep_limit
    #     eval_return = eval_rews.sum()
    # 
    #     with lock:
    #         logger.info('Eval result: task={} return={:.3f} length={}'.format(task_id, eval_return, eval_length))
    # 
    #     result_queue.put(Result(
    #         worker_id=worker_id,
    #         noise_inds_n=None,
    #         returns_n2=None,
    #         signreturns_n2=None,
    #         lengths_n2=None,
    #         eval_return=eval_return,
    #         eval_length=eval_length,
    #         ob_sum=None,
    #         ob_sumsq=None,
    #         ob_count=None,
    #         task_id=task_id
    #     ))

    # Rollouts with noise
    noise_inds, returns, signreturns, lengths = [], [], [], []
    task_ob_stat = RunningStat(env.observation_space.shape, eps=0.)  # eps=0 because we're incrementing only
    
    #while not noise_inds or time.time() - task_tstart < min_task_runtime:
    
    for _ in range(num_jobs):

        # ------------- Noise sample -------------------------------
        noise_idx = noise.sample_index(rs, num_params)
        epsilon = config.noise_stdev * noise.get(noise_idx, num_params)

        # Evaluate the sampled noise positive
        set_from_flat(model, theta + epsilon)
        rews_pos, len_pos = rollout_and_update_ob_stat(env, model, rs=rs, task_ob_stat=task_ob_stat, ob_mean=ob_mean, ob_std=ob_std)

        # rews_pos, len_pos = rollout_and_update_ob_stat(
        #     policy, env, task_data.timestep_limit, rs, task_ob_stat, config.calc_obstat_prob)
        
        if optimizations.mirrored_sampling:
            # Evaluate the sample noise negative
            set_from_flat(model, theta - epsilon)
            rews_neg, len_neg = rollout_and_update_ob_stat(env, model, rs=rs, task_ob_stat=task_ob_stat, ob_mean=ob_mean, ob_std=ob_std)

        # rews_neg, len_neg = rollout_and_update_ob_stat(
        #     policy, env, task_data.timestep_limit, rs, task_ob_stat, config.calc_obstat_prob)
        
        # Gather results
        noise_inds.append(noise_idx)
        returns.append([rews_pos.sum()])
        signreturns.append([np.sign(rews_pos).sum()])
        lengths.append([len_pos])
        
        if optimizations.mirrored_sampling:
            returns[-1].append(rews_neg.sum())
            signreturns[-1].append(np.sign(rews_neg).sum())
            lengths[-1].append(len_neg)
        
    # result_queue.put(Result(
    #     worker_id=worker_id,
    #     noise_inds_n=np.array(noise_inds),
    #     returns_n2=np.array(returns, dtype=np.float32),
    #     signreturns_n2=np.array(signreturns, dtype=np.float32),
    #     lengths_n2=np.array(lengths, dtype=np.int32),
    #     eval_return=None,
    #     eval_length=None,
    #     ob_sum=None if task_ob_stat.count == 0 else task_ob_stat.sum,
    #     ob_sumsq=None if task_ob_stat.count == 0 else task_ob_stat.sumsq,
    #     ob_count=task_ob_stat.count,
    #     task_id=task_id
    # ))
    print("PID " + str(os.getpid()) + ": " + "Returned result")
    result = Result(
        noise_inds=np.array(noise_inds),
        returns=np.array(returns, dtype=np.float32),
        signreturns=np.array(signreturns, dtype=np.float32),
        lengths=np.array(lengths, dtype=np.int32),
        eval_return=None,
        eval_length=None,
        ob_sum=None if task_ob_stat.count == 0 else task_ob_stat.sum,
        ob_sumsq=None if task_ob_stat.count == 0 else task_ob_stat.sumsq,
        ob_count=task_ob_stat.count,
        task_id = 0
    )
    
    return result

In [ ]:
def itergroups(items, group_size):
    assert group_size >= 1
    group = []
    for x in items:
        group.append(x)
        if len(group) == group_size:
            yield tuple(group)
            del group[:]
    if group:
        yield tuple(group)
        
def batched_weighted_sum(weights, vecs, batch_size):
    total = 0.
    num_items_summed = 0
    for batch_weights, batch_vecs in zip(itergroups(weights, batch_size), itergroups(vecs, batch_size)):
        assert len(batch_weights) == len(batch_vecs) <= batch_size
        total += np.dot(np.asarray(batch_weights, dtype=np.float32), np.asarray(batch_vecs, dtype=np.float32))
        num_items_summed += len(batch_weights)
    return total, num_items_summed

## Optimization: Fitness shaping with a rank transformation

In [ ]:
def compute_ranks(x):
    """
    Returns ranks in [0, len(x))
    Note: This is different from scipy.stats.rankdata, which returns ranks in [1, len(x)].
    """
    assert x.ndim == 1
    ranks = np.empty(len(x), dtype=int)
    ranks[x.argsort()] = np.arange(len(x))
    return ranks


def compute_centered_ranks(x):
    y = compute_ranks(x.ravel()).reshape(x.shape).astype(np.float32)
    return y

# Master

In [ ]:
env = gym.make(config.env_id)
rs = np.random.RandomState()

ob_stat = RunningStat(
    env.observation_space.shape,
    eps=1e-2  # eps to prevent dividing by zero at the beginning when computing mean/stdev
)

tslimit, incr_tslimit_threshold, tslimit_incr_ratio = None, None, None
adaptive_tslimit = False


episodes_so_far = 0
timesteps_so_far = 0
tstart = time.time()

task_counter = 0

assert config.num_workers != 0

num_jobs_per_worker = [int(config.population_size / config.num_workers)] * config.num_workers

mod = config.population_size % config.num_workers
i = 0
while mod > 0:
    num_jobs_per_worker[i] += 1
    mod -= 1
    i += 1
    
assert len(num_jobs_per_worker) == config.num_workers
generation_counter = 0

theta = get_flat(theta)
old_theta = theta

while True:
    print("----------------------GENERATION: " + str(generation_counter) + "------------------------------------")
    logger.info("Generation {} started".format(generation_counter))
    
    step_tstart = time.time() 
            
    #assert theta.dtype == np.float32

    # Task counter is used to recognize false tasks from previous iterations later
    curr_task_id = task_counter
    task_counter += 1
    
    # Start workers
    
    workers = []
    results = []
    
    with multiprocessing.Pool(processes=config.num_workers) as pool:
    
        print("PID " + str(os.getpid()) + ": " + "Waiting for results")
        for i in num_jobs_per_worker:
            results.append(pool.apply_async(func=run_worker, args=(i, theta, ob_stat.mean, ob_stat.std)))

        for i in range(len(results)):
            results[i] = results[i].get()

    # Pop off results for the current task
    curr_task_results, eval_rets, eval_lens, worker_ids = [], [], [], []
    num_results_skipped, num_episodes_popped, num_timesteps_popped, ob_count_this_batch = 0, 0, 0, 0
   #while num_episodes_popped < config.episodes_per_batch:
    for result in results:
        assert isinstance(result, Result)
        # task_id = result.task_id
        # assert isinstance(task_id, int)

        # assert (result.eval_return is None) == (result.eval_length is None)
        # worker_ids.append(result.worker_id)
        # 
        # if result.eval_length is not None:
        #     # This was an eval job
        #     episodes_so_far += 1
        #     timesteps_so_far += result.eval_length
        #     # Store the result only for current tasks
        #     if task_id == curr_task_id:
        #         eval_rets.append(result.eval_return)
        #         eval_lens.append(result.eval_length)
        # else:

        
        assert result.noise_inds_n.ndim == 1
        if optimizations.mirrored_sampling:
            assert result.returns.shape == result.lengths.shape == (len(result.noise_inds), 2)
        else:
            assert result.returns.shape == result.lengths.shape == (len(result.noise_inds), 1)
        assert result.returns.dtype == np.float32
        
        # Update counts
        result_num_eps = result.lengths.size
        result_num_timesteps = result.lengths.sum()
        episodes_so_far += result_num_eps
        timesteps_so_far += result_num_timesteps
        # Store results only for current tasks
        curr_task_results.append(result)
        num_episodes_popped += result_num_eps
        num_timesteps_popped += result_num_timesteps
        # Update ob stats
        if result.ob_count > 0:
            ob_stat.increment(result.ob_sum, result.ob_sumsq, result.ob_count)
            ob_count_this_batch += result.ob_count

    # Compute skip fraction
    #frac_results_skipped = num_results_skipped / (num_results_skipped + len(curr_task_results))
    # if num_results_skipped > 0:
    #     logger.warning('Skipped {} out of date results ({:.2f}%)'.format(
    #         num_results_skipped, 100. * frac_results_skipped))
    
    print("Gathered results")

    # Assemble results
    noise_inds = np.concatenate([r.noise_inds for r in curr_task_results])
    returns = np.concatenate([r.returns for r in curr_task_results])
    lengths = np.concatenate([r.lengths for r in curr_task_results])
    assert noise_inds.shape[0] == returns.shape[0] == lengths.shape[0]
    
    # Process returns
    if optimizations.fitness_shaping:
        if config.return_proc_mode == 'centered_rank':
            proc_returns = compute_centered_ranks(returns)
        # sign and centered_sign_rank are obviously only useful in combination with mirrored sampling
        elif config.return_proc_mode == 'sign':
            proc_returns = np.concatenate([r.signreturns for r in curr_task_results])
        elif config.return_proc_mode == 'centered_sign_rank':
            proc_returns = compute_centered_ranks(np.concatenate([r.signreturns for r in curr_task_results]))
    else:
        proc_returns = returns
    
    if optimizations.mirrored_sampling:
        # Calculate the difference between the rewards sampled with the positive and negative noise
        proc_returns = proc_returns[:, 0] - proc_returns[:, 1]
    else:
        proc_returns = proc_returns.ravel()
    
    g, count = batched_weighted_sum(
        proc_returns,
        (noise.get(idx, num_params) for idx in noise_inds),
        batch_size=500
    )
    
    assert g.shape == (num_params,) and g.dtype == np.float32 and count == len(noise_inds)
    
    # Update with the approximated gradient
    g /= returns.size
    
    if optimizations.neural_network_optimizer:
        theta, _ = optimizer.update(theta, -g + config.l2coeff * theta)
    else:
        theta += ((config.learning_rate / config.noise_stdev) * g)
    

    # Update ob stat (we're never running the policy in the master, but we might be snapshotting the policy)
    # if policy.needs_ob_stat:
    #     policy.set_ob_stat(ob_stat.mean, ob_stat.std)

    # Update number of steps to take
    # if adaptive_tslimit and (lengths_n2 == tslimit).mean() >= incr_tslimit_threshold:
    #     old_tslimit = tslimit
    #     tslimit = int(tslimit_incr_ratio * tslimit)
    #     logger.info('Increased timestep limit from {} to {}'.format(old_tslimit, tslimit))

    step_tend = time.time()
    
    logger.info("EpRewMean {}".format(returns.mean()))
    logger.info("EpRewStd {}".format(returns.std()))
    logger.info("EpLenMean {}".format(lengths.mean()))
    # 
    # tlogger.record_tabular("EvalEpRewMean", np.nan if not eval_rets else np.mean(eval_rets))
    # tlogger.record_tabular("EvalEpRewStd", np.nan if not eval_rets else np.std(eval_rets))
    # tlogger.record_tabular("EvalEpLenMean", np.nan if not eval_rets else np.mean(eval_lens))
    # tlogger.record_tabular("EvalPopRank", np.nan if not eval_rets else (
    #     np.searchsorted(np.sort(returns_n2.ravel()), eval_rets).mean() / returns_n2.size))
    # tlogger.record_tabular("EvalEpCount", len(eval_rets))
    # 
    # tlogger.record_tabular("Norm", float(np.square(policy.get_trainable_flat()).sum()))
    # tlogger.record_tabular("GradNorm", float(np.square(g).sum()))
    # tlogger.record_tabular("UpdateRatio", float(update_ratio))
    # 
    logger.info("EpisodesThisIter {}".format(lengths.size))
    logger.info("EpisodesSoFar {}".format(episodes_so_far))
    logger.info("TimestepsThisIter {}".format(lengths.sum()))
    logger.info("TimestepsSoFar {}".format(timesteps_so_far))
    # 
    # num_unique_workers = len(set(worker_ids))
    # tlogger.record_tabular("UniqueWorkers", num_unique_workers)
    # tlogger.record_tabular("UniqueWorkersFrac", num_unique_workers / len(worker_ids))
    # tlogger.record_tabular("ResultsSkippedFrac", frac_results_skipped)
    logger.info("ObCount {}".format(ob_count_this_batch))
    # 
    logger.info("TimeElapsedThisIter {}".format(step_tend - step_tstart))
    logger.info("TimeElapsed {}".format(step_tend - tstart))

    # tlogger.dump_tabular()

    if config.snapshot_freq != 0 and generation_counter % config.snapshot_freq == 0:
        from multiprocessing import Process
        
        p = Process(target=create_model, args=(theta, 
                                               config.env_id + "_Generation_" + str(generation_counter), 
                                               save_directory + 'snapshot_{:05d}'.format(generation_counter) + ".h5"))
        p.start()
        p.join()
        
        print("Saved model in generation {}".format(generation_counter))
        logger.info("Saved model to {}".format(save_directory))
            
    generation_counter+= 1
    
    logger.info("-----------------------------------------------------------------")

In [ ]:
for i in range(10):
    r = (-1) + (i * 2/9)
    print(r)